In [7]:
conda install openpyxl

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import glob
import openpyxl

## Eolicas

In [54]:
rebelde =pd.read_csv('/Users/ctalamilla/Desktop/TacaTaca/data/Eólica 2/csv/20682654.csv', skiprows=0, sep='\t').set_axis(['fecha','hora','Vel_kmh', 'Vel_Raf_kmh', 'Dir_viento_grd'], axis='columns')
rebelde

,fecha,hora,Vel_kmh,Vel_Raf_kmh,Dir_viento_grd
0,03/13/20,08:21:07,2.412,3.600,33.7
1,03/13/20,08:22:07,1.188,2.412,35.1
2,03/13/20,08:23:07,0.000,1.188,35.1
3,03/13/20,08:24:07,0.000,0.000,35.1
4,03/13/20,08:25:07,0.000,1.188,35.1
...,...,...,...,...,...
59417,04/23/20,14:38:07,16.848,20.448,332.7
59418,04/23/20,14:39:07,15.624,20.448,332.7
59419,04/23/20,14:40:07,18.036,22.860,351.0
59420,04/23/20,14:41:07,10.836,16.848,338.3


In [55]:
rebelde['fecha'] = rebelde['fecha']+ ' ' + rebelde['hora']
rebelde

,fecha,hora,Vel_kmh,Vel_Raf_kmh,Dir_viento_grd
0,03/13/20 08:21:07,08:21:07,2.412,3.600,33.7
1,03/13/20 08:22:07,08:22:07,1.188,2.412,35.1
2,03/13/20 08:23:07,08:23:07,0.000,1.188,35.1
3,03/13/20 08:24:07,08:24:07,0.000,0.000,35.1
4,03/13/20 08:25:07,08:25:07,0.000,1.188,35.1
...,...,...,...,...,...
59417,04/23/20 14:38:07,14:38:07,16.848,20.448,332.7
59418,04/23/20 14:39:07,14:39:07,15.624,20.448,332.7
59419,04/23/20 14:40:07,14:40:07,18.036,22.860,351.0
59420,04/23/20 14:41:07,14:41:07,10.836,16.848,338.3


In [56]:
rebelde = rebelde.drop(['hora'], axis=1)
rebelde

,fecha,Vel_kmh,Vel_Raf_kmh,Dir_viento_grd
0,03/13/20 08:21:07,2.412,3.600,33.7
1,03/13/20 08:22:07,1.188,2.412,35.1
2,03/13/20 08:23:07,0.000,1.188,35.1
3,03/13/20 08:24:07,0.000,0.000,35.1
4,03/13/20 08:25:07,0.000,1.188,35.1
...,...,...,...,...
59417,04/23/20 14:38:07,16.848,20.448,332.7
59418,04/23/20 14:39:07,15.624,20.448,332.7
59419,04/23/20 14:40:07,18.036,22.860,351.0
59420,04/23/20 14:41:07,10.836,16.848,338.3


In [57]:
rebelde['fecha'] = pd.to_datetime(rebelde.fecha)
rebelde

,fecha,Vel_kmh,Vel_Raf_kmh,Dir_viento_grd
0,2020-03-13 08:21:07,2.412,3.600,33.7
1,2020-03-13 08:22:07,1.188,2.412,35.1
2,2020-03-13 08:23:07,0.000,1.188,35.1
3,2020-03-13 08:24:07,0.000,0.000,35.1
4,2020-03-13 08:25:07,0.000,1.188,35.1
...,...,...,...,...
59417,2020-04-23 14:38:07,16.848,20.448,332.7
59418,2020-04-23 14:39:07,15.624,20.448,332.7
59419,2020-04-23 14:40:07,18.036,22.860,351.0
59420,2020-04-23 14:41:07,10.836,16.848,338.3


In [52]:
rebelde.to_csv('../data/Eólica 2/all_files/20682654_fix.csv', sep=',')

In [11]:
archivosAll = np.array(glob.glob('../data/Eólica 1/all_files/*'))
rutas = pd.DataFrame({'files': archivosAll})
rutas

,files
0,../data/Eólica 1/all_files/E1_22-10-2021.csv
1,../data/Eólica 1/all_files/E1_20-04-22.csv
2,../data/Eólica 1/all_files/20682654.xlsx
3,../data/Eólica 1/all_files/E1_17-03-2022.csv
4,../data/Eólica 1/all_files/E1_29-03-2022.csv
5,../data/Eólica 1/all_files/E1_17-03-2022.xlsx
6,../data/Eólica 1/all_files/E1_14-12-2021.csv
7,../data/Eólica 1/all_files/20682650.csv
8,../data/Eólica 1/all_files/20682654.csv
9,../data/Eólica 1/all_files/E1_22-10-2021.xlsx


In [74]:
def cleanEolica(path, estacion):
        archivosAll = np.array(glob.glob(path +'/*'))
        rutas = pd.DataFrame({'files': archivosAll})


        eolica1 = []
        for index, value in rutas.files.str.contains('csv').items():
            if (value== True):
                #print(rutas.files[index])
                file_readed = pd.read_csv(rutas.files[index], skiprows=1, usecols=[1,2,3,4]).set_axis(['fecha','Vel_kmh', 'Vel_Raf_kmh', 'Dir_viento_grd'], axis='columns')
                file_readed['fecha'] = pd.to_datetime(file_readed['fecha'] )
                file_readed['estacion']= estacion
                eolica1.append(file_readed)
                #print(file_readed)
            else:
                #print(rutas.files[index])
                file_readed = pd.read_excel(rutas.files[index], skiprows=1, usecols=[1,2,3,4]).set_axis(['fecha','Vel_kmh', 'Vel_Raf_kmh', 'Dir_viento_grd'], axis='columns')
                file_readed['fecha'] = pd.to_datetime(file_readed['fecha'] )
                file_readed['estacion']= estacion
                eolica1.append(file_readed)
                #print(file_readed)

        return pd.concat(eolica1).drop_duplicates(['fecha']).reset_index(drop=True)

In [76]:
EstacionEolica1 = cleanEolica('../data/Eólica 1/all_files', 'Eolica1')
EstacionEolica1

,fecha,Vel_kmh,Vel_Raf_kmh,Dir_viento_grd,estacion
0,2021-06-08 16:34:05,28.872,42.084,206.4,Eolica1
1,2021-06-08 16:35:05,32.472,39.672,210.6,Eolica1
2,2021-06-08 16:36:05,27.648,36.072,226.0,Eolica1
3,2021-06-08 16:37:05,30.060,38.484,219.0,Eolica1
4,2021-06-08 16:38:05,24.048,27.648,219.0,Eolica1
...,...,...,...,...,...
336539,2021-12-12 23:55:00,32.472,37.260,220.4,Eolica1
336540,2021-12-12 23:56:00,34.848,39.672,220.4,Eolica1
336541,2021-12-12 23:57:00,31.248,34.848,227.4,Eolica1
336542,2021-12-12 23:58:00,27.648,31.248,223.2,Eolica1


In [78]:
EstacionEolica1.duplicated(['fecha']).sum()

0

In [77]:
EstacionEolica2 = cleanEolica('../data/Eólica 2/all_files', 'Eolica2')
EstacionEolica2

,fecha,Vel_kmh,Vel_Raf_kmh,Dir_viento_grd,estacion
0,2022-03-17 16:48:48,30.060,43.272,158.6,Eolica2
1,2022-03-17 16:53:48,31.248,49.284,160.0,Eolica2
2,2022-03-17 16:58:48,32.472,46.872,165.7,Eolica2
3,2022-03-17 17:03:48,32.472,48.096,162.9,Eolica2
4,2022-03-17 17:08:48,15.624,24.048,162.9,Eolica2
...,...,...,...,...,...
241448,2022-01-07 07:58:27,-888.880,-888.880,-888.9,Eolica2
241449,2022-01-07 07:59:27,-888.880,-888.880,-888.9,Eolica2
241450,2022-01-07 08:00:27,-888.880,-888.880,-888.9,Eolica2
241451,2022-01-07 08:01:27,-888.880,-888.880,-888.9,Eolica2


In [79]:
EstacionEolica2.duplicated(['fecha']).sum()

0

In [82]:
EstacionEolica1.to_csv('Eolica1_cleaned.csv', sep=',')
EstacionEolica2.to_csv('Eolica2_cleaned.csv', sep=',')

## Davis 

In [149]:
def cleanDavis(ruta, estacion):
        archivosAll = np.array(glob.glob(ruta +'/*'))
        rutas = pd.DataFrame({'files': archivosAll})
        rutas

        davis = []
        for index, value in rutas.files.str.contains('csv').items():
                #print(rutas.files[index])
                colNames = ['fecha', 
                            'hora', 
                            'tempAmbiente-c', 
                            'tempMaxima-c', 
                            'tempMinima-c',
                            'humRelativa-%', 
                            'puntoRocio-c', 
                            'velViento-mps',
                            'dirViento-cuad', 
                            'relViento',
                            'velVientoMaxima-mps', 
                            'dirVientoPredominante-cuad', 
                            'sensTermina-c',
                            'indiceCalor', 
                            'indiceTHW',
                            'indiceTHSW', 
                            'presionATM',
                            'precipitacion-mm', 
                            'intPrecipitacion-mmh',
                            'radiacionSolar',
                            'energiaSolar', 
                            'radiacionSolarMaxima', 
                            'heatD-D', 
                            'coolD-D',
                            'tempInterior-c',
                            'humRelativaInterior-%', 
                            'puntoRocioInterior-c',
                            'indiceCalorInterior',
                            'evapotranspiracion-mm',
                            'windSamp', 
                            'windTx',
                            'ISSRecept',
                            'arcInt']
                file_readed = pd.read_csv(rutas.files[index], skiprows=2, sep=';', low_memory=False).set_axis(colNames, axis='columns')
                file_readed['Estacion'] = estacion
                file_readed['fechaHora'] = pd.to_datetime(file_readed['fecha']+ ' ' + file_readed['hora'])
                
                davis.append(file_readed)
                #print(file_readed)
        return pd.concat(davis).drop_duplicates(['fecha', 'hora']).reset_index(drop=True)

In [154]:
Davis_TacaTaca_cleaned = cleanDavis('../data/EM_Taca Taca/all_files', 'Estacion Taca Taca')

In [155]:
Davis_TacaTaca_cleaned

,fecha,hora,tempAmbiente-c,tempMaxima-c,tempMinima-c,humRelativa-%,puntoRocio-c,velViento-mps,dirViento-cuad,relViento,...,humRelativaInterior-%,puntoRocioInterior-c,indiceCalorInterior,evapotranspiracion-mm,windSamp,windTx,ISSRecept,arcInt,Estacion,fechaHora
0,30/9/11,10:00,10.4,10.4,9.3,15,-15.2,8.0,SE,2.01,...,14,-14.2,10.5,0.3,351,1,100.0,15,Estacion Taca Taca,2011-09-30 10:00:00
1,30/9/11,10:15,10.7,10.8,10.4,14,-15.8,8.0,SSE,2.01,...,14,-14.4,10.3,0,351,1,100.0,15,Estacion Taca Taca,2011-09-30 10:15:00
2,30/9/11,10:30,10.8,11,10.7,13,-16.6,11.3,SSE,2.82,...,14,-14.5,10.3,0,352,1,100.0,15,Estacion Taca Taca,2011-09-30 10:30:00
3,30/9/11,10:45,10.7,10.9,10.7,14,-15.8,11.3,SSE,2.82,...,14,-14.6,10.2,0,351,1,100.0,15,Estacion Taca Taca,2011-09-30 10:45:00
4,30/9/11,11:00,11.6,11.6,10.7,13,-15.9,9.7,SSE,2.41,...,13,-15.2,10.4,0.48,351,1,100.0,15,Estacion Taca Taca,2011-09-30 11:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253890,30/4/21,4:00 a.m.,0.5,1.2,-0.4,19,-20.5,4.8,NO,9.66,...,30,-17.6,-2.9,0.13,2805,1,100.0,120,Estacion Taca Taca,2021-04-30 04:00:00
253891,30/4/21,6:00 a.m.,-1.5,-0.4,-2.5,24,-19.5,8.0,NO,16.09,...,32,-17.8,-4.0,0.13,2807,1,100.0,120,Estacion Taca Taca,2021-04-30 06:00:00
253892,30/4/21,8:00 a.m.,-2.6,-1.0,-3.6,24,-20.4,8.0,ONO,16.09,...,32,-18.8,-5.1,0.10,2804,1,100.0,120,Estacion Taca Taca,2021-04-30 08:00:00
253893,30/4/21,10:00 a.m.,-1.1,2.9,-4.1,18,-22.4,3.2,N,6.44,...,33,-17.9,-4.4,0.18,2806,1,100.0,120,Estacion Taca Taca,2021-04-30 10:00:00


In [156]:
Davis_Vendaval_cleaned=cleanDavis('../data/EM_Vendaval/all_files', 'Estacion Vendaval')

In [157]:
Davis_Vendaval_cleaned

,fecha,hora,tempAmbiente-c,tempMaxima-c,tempMinima-c,humRelativa-%,puntoRocio-c,velViento-mps,dirViento-cuad,relViento,...,humRelativaInterior-%,puntoRocioInterior-c,indiceCalorInterior,evapotranspiracion-mm,windSamp,windTx,ISSRecept,arcInt,Estacion,fechaHora
0,6/6/21,14:00,1.9,2.1,0.2,19,-19.3,27.4,WNW,54.72,...,19,-15.4,5.6,0.84,2801,1,100.0,120,Estacion Vendaval,2021-06-06 14:00:00
1,6/6/21,16:00,1.6,2.4,1.3,21,-18.4,20.9,NW,41.84,...,17,-14.6,8.0,0.74,2806,1,100.0,120,Estacion Vendaval,2021-06-06 16:00:00
2,6/6/21,18:00,0.2,1.8,0.2,23,-18.6,37.0,WNW,74.03,...,17,-18.6,3.3,0.61,2802,1,100.0,120,Estacion Vendaval,2021-06-06 18:00:00
3,6/6/21,20:00,-3.1,0.2,-3.1,29,-18.6,37.0,WNW,74.03,...,19,-24.6,-4.9,0.33,2802,1,100.0,120,Estacion Vendaval,2021-06-06 20:00:00
4,6/6/21,22:00,-4.2,-3.0,-4.2,31,-18.8,17.7,N,35.41,...,22,-25.0,-7.3,0.15,2807,1,100.0,120,Estacion Vendaval,2021-06-06 22:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4295,30/5/22,10:00,-3.7,-3.7,-7.6,45,-13.9,25.7,S,51.50,...,38,-16.3,-4.4,0.30,2806,1,100.0,120,Estacion Vendaval,2022-05-30 10:00:00
4296,30/5/22,12:00,-0.8,-0.8,-3.8,28,-17.1,41.8,S,83.69,...,32,-14.6,-0.4,0.74,2805,1,100.0,120,Estacion Vendaval,2022-05-30 12:00:00
4297,30/5/22,14:00,0.5,0.9,-1.0,22,-18.8,54.7,S,109.44,...,24,-14.1,3.8,1.07,2802,1,100.0,120,Estacion Vendaval,2022-05-30 14:00:00
4298,30/5/22,16:00,0.7,1.6,0.4,19,-20.3,56.3,S,112.65,...,21,-16.4,3.0,0.94,2807,1,100.0,120,Estacion Vendaval,2022-05-30 16:00:00


In [ ]:
Davis_TacaTaca_cleaned.to_csv('Davis_TacaTaca.csv', sep=',')
Davis_Vendaval_cleaned.to_csv('DavisVendaval.csv', sep=',')